## POS tagging using modified Viterbi

### Data Preparation

In [1]:
#Importing libraries
import nltk
import numpy as np
import pandas as pd
import pprint,time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
import math

In [2]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [3]:
#converting the list of sents to list of words(word,pos tuples)
tagged_words=[tup for sent in nltk_data for tup in sent]
print(len(tagged_words))
tagged_words[:10]

100676


[('Pierre', 'NOUN'),
 ('Vinken', 'NOUN'),
 (',', '.'),
 ('61', 'NUM'),
 ('years', 'NOUN'),
 ('old', 'ADJ'),
 (',', '.'),
 ('will', 'VERB'),
 ('join', 'VERB'),
 ('the', 'DET')]

In [4]:
# splitting into train and test set
random.seed(1234)
train_set,test_set=train_test_split(nltk_data,test_size=0.05)
print(len(train_set))
print(len(test_set))

3718
196


In [5]:
# getting list of train tagged words
train_tagged_words=[tup for sent in train_set for tup in sent]
len(train_tagged_words)

95174

In [6]:
#tokens
tokens=[pair[0] for pair in train_tagged_words]

In [7]:
#vocabulary
V=set(tokens)
print(len(V))
#number of tags
T=set([pair[1] for pair in train_tagged_words])
len(T)

12038


12

In [8]:
#Emission Probabilities
#computing p(w/t) and storing in TxV matrix
t=len(T)
v=len(V)
w_given_t=np.zeros((t,v))

In [9]:
def word_given_tag(word,tag,train_bag=train_tagged_words):
    tag_list=[pair for pair in train_bag if pair[1]==tag]
    count_tag=len(tag_list)
    w_given_tag_list=[pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag=len(w_given_tag_list)
    return (count_w_given_tag,count_tag)

In [10]:
# Transition probabilities
def t2_given_t1(t2,t1,train_bag=train_tagged_words):
    tags=[pair[1] for pair in train_bag]
    count_t1=len([t for t in tags if t==t1])
    count_t2_t1=0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1]==t2:
            count_t2_t1+=1
    return (count_t2_t1,count_t1)

In [11]:
# creating txt transition matrix of tags
# each column is t2 , each row is t1
tags_matrix=np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [12]:
#convert the matrix to a df for better readability
tags_df=pd.DataFrame(tags_matrix,columns=list(T),index=list(T))
tags_df


,.,NUM,X,PRON,NOUN,ADV,VERB,DET,ADP,ADJ,CONJ,PRT
.,0.092545,0.079000,0.027455,0.067273,0.221000,0.052727,0.089091,0.174273,0.091273,0.044455,0.058364,0.002455
NUM,0.119684,0.184083,0.209903,0.001519,0.353888,0.003038,0.017922,0.003341,0.035237,0.032807,0.013670,0.024909
X,0.163706,0.002880,0.074252,0.056009,0.061130,0.026244,0.204193,0.055529,0.141783,0.017123,0.009762,0.187390
PRON,0.039847,0.007663,0.093103,0.008046,0.211111,0.035249,0.483525,0.009962,0.021073,0.073946,0.004981,0.011494
NOUN,0.237743,0.009037,0.029599,0.004647,0.264781,0.017123,0.147227,0.013135,0.177996,0.012110,0.042624,0.043978
ADV,0.135198,0.030636,0.022977,0.014985,0.031968,0.078921,0.349317,0.068265,0.117216,0.129204,0.006660,0.014652
VERB,0.034767,0.022867,0.217547,0.035545,0.110523,0.081434,0.168469,0.135490,0.091545,0.065178,0.005367,0.031267
DET,0.017825,0.021705,0.044744,0.003516,0.640718,0.012368,0.038559,0.005699,0.009094,0.205044,0.000485,0.000243
ADP,0.039195,0.063290,0.034376,0.069287,0.325016,0.013493,0.008246,0.321161,0.017134,0.106340,0.000964,0.001499
ADJ,0.064340,0.020179,0.020675,0.000662,0.698975,0.004797,0.012239,0.005127,0.078234,0.066159,0.017367,0.011247


### Build the vanilla Viterbi based POS tagger

In [14]:
# Viterbi Heuristic
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1] 
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))



In [15]:
# Let's test our Viterbi algorithm on a few sample sentences of test dataset

random.seed(1234)

# choose random 5 sents
rndom = [random.randint(1,len(test_set)) for x in range(5)]

# list of sents
test_run = [test_set[i] for i in rndom]

# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]
print(test_run)

[[('The', 'DET'), ('Tokyo', 'NOUN'), ('Stock', 'NOUN'), ('Price', 'NOUN'), ('Index', 'NOUN'), ('-LRB-', '.'), ('Topix', 'NOUN'), ('-RRB-', '.'), ('of', 'ADP'), ('all', 'DET'), ('issues', 'NOUN'), ('listed', 'VERB'), ('*', 'X'), ('in', 'ADP'), ('the', 'DET'), ('First', 'NOUN'), ('Section', 'NOUN'), (',', '.'), ('which', 'DET'), ('*T*-1', 'X'), ('gained', 'VERB'), ('16.05', 'NUM'), ('points', 'NOUN'), ('Tuesday', 'NOUN'), (',', '.'), ('was', 'VERB'), ('down', 'ADV'), ('1.46', 'NUM'), ('points', 'NOUN'), (',', '.'), ('or', 'CONJ'), ('0.05', 'NUM'), ('%', 'NOUN'), (',', '.'), ('at', 'ADP'), ('2691.19', 'NUM'), ('.', '.')], [('``', '.'), ('It', 'PRON'), ("'s", 'VERB'), ('a', 'DET'), ('cosmetic', 'ADJ'), ('move', 'NOUN'), (',', '.'), ("''", '.'), ('said', 'VERB'), ('*T*-1', 'X'), ('Jonathan', 'NOUN'), ('S.', 'NOUN'), ('Gelles', 'NOUN'), ('of', 'ADP'), ('Wertheim', 'NOUN'), ('Schroder', 'NOUN'), ('&', 'CONJ'), ('Co', 'NOUN'), ('.', '.')], [('The', 'DET'), ('flow', 'NOUN'), ('of', 'ADP'), ('Ja

In [16]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)
print(tagged_seq)
#print(test_run_base)

Time taken in seconds:  32.10798645019531
[('The', 'DET'), ('Tokyo', 'NOUN'), ('Stock', 'NOUN'), ('Price', 'NOUN'), ('Index', 'NOUN'), ('-LRB-', '.'), ('Topix', '.'), ('-RRB-', '.'), ('of', 'ADP'), ('all', 'DET'), ('issues', 'NOUN'), ('listed', 'VERB'), ('*', 'X'), ('in', 'ADP'), ('the', 'DET'), ('First', 'NOUN'), ('Section', 'NOUN'), (',', '.'), ('which', 'DET'), ('*T*-1', 'X'), ('gained', 'VERB'), ('16.05', '.'), ('points', 'NOUN'), ('Tuesday', 'NOUN'), (',', '.'), ('was', 'VERB'), ('down', 'ADV'), ('1.46', '.'), ('points', 'NOUN'), (',', '.'), ('or', 'CONJ'), ('0.05', '.'), ('%', 'NOUN'), (',', '.'), ('at', 'ADP'), ('2691.19', '.'), ('.', '.'), ('``', '.'), ('It', 'PRON'), ("'s", 'VERB'), ('a', 'DET'), ('cosmetic', 'NOUN'), ('move', 'NOUN'), (',', '.'), ("''", '.'), ('said', 'VERB'), ('*T*-1', 'X'), ('Jonathan', 'NOUN'), ('S.', 'NOUN'), ('Gelles', '.'), ('of', 'ADP'), ('Wertheim', '.'), ('Schroder', '.'), ('&', 'CONJ'), ('Co', 'NOUN'), ('.', '.'), ('The', 'DET'), ('flow', 'NOUN'), (

In [17]:
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 

accuracy = len(check)/len(tagged_seq)
accuracy

0.9214285714285714

In [18]:
# reading smaple test file
f=open('Test_sentences.txt','r')
sentences=f.read()

In [19]:
# passing sentences to vanila viterbi algo
words =word_tokenize(sentences)
start=time.time()
tagged_sample_test_1=Viterbi(words)
end=time.time()
difference=end-start

In [20]:
print(tagged_sample_test_1)

[('Android', '.'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', '.'), ('.', '.'), ('Android', '.'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', '.'), ('worldwide', '.'), ('on', 'ADP'), ('smartphones', '.'), ('since', 'ADP'), ('2011', '.'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', '.'), ('.', '.'), ('Google', '.'), ('and', 'CONJ'), ('Twitter', '.'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', '.'), ('that', 'DET'), ('gave', 'VERB'), ('Google', '.'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', '.'), ("'s", 'VERB'), ('firehose', '.'), ('.', '.'), ('Twitter', '.'), ('is', 'VERB'), ('an', 'DET'), ('online', '.'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DET'), ('users', 'NOUN'), ('post', 'NOUN'), ('and', 

 As we can see that Android , OS , Google  and many more are tagged incorrectly

### Solve the problem of unknown words

In [21]:
#unique tags are there in the corpus
tags=[pair[1] for pair in tagged_words]
unique_tags=set(tags)
len(unique_tags)

12

In [22]:
# lets check the most common_tags 
from collections import Counter
tag_counts=Counter(tags)
tag_counts

Counter({'NOUN': 28867,
         '.': 11715,
         'NUM': 3546,
         'ADJ': 6397,
         'VERB': 13564,
         'DET': 8725,
         'ADP': 9857,
         'CONJ': 2265,
         'X': 6613,
         'ADV': 3171,
         'PRT': 3219,
         'PRON': 2737})

So the most common tags is Noun , So lets assign tag Noun to unknown words 

In [23]:
# modified viterbi algo which will assign tag NOUN to unknown words

def Viterbi_POS(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1] 
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        if pmax!=0.0:
            # getting state for which probability is maximum
            state_max = T[p.index(pmax)]
        else:
            # assigning the most occuring POS to unknown words
            state_max='NOUN'
        state.append(state_max)
    return list(zip(words, state))




In [24]:
# lets check on test sample sentences 
tagged_seq_POS=Viterbi_POS(words)
print(tagged_seq_POS)

[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.'), ('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'NOUN'), ('since', 'ADP'), ('2011', 'NOUN'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NOUN'), ('.', '.'), ('Google', 'NOUN'), ('and', 'CONJ'), ('Twitter', 'NOUN'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'NOUN'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'NOUN'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'NOUN'), ("'s", 'PRT'), ('firehose', 'NOUN'), ('.', '.'), ('Twitter', 'NOUN'), ('is', 'VERB'), ('an', 'DET'), ('online', 'NOUN'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DET')

 As we can see that for unknow words it has tagged the most occuring tag which is NOUN

In [25]:
# let apply rule based tagging and modify viterbi algo
# specify patterns for tagging
# example from the NLTK book
patterns = [
    (r'(.*ing|.*ed)$', 'VERB'),              
    (r'\d+','NUM'),
    (r'.*ity$','ADV'),
    (r'.*', 'NOUN'),
    
]
regexp_tagger = nltk.RegexpTagger(patterns)
# help(regexp_tagger)
rule_based_tagger = nltk.RegexpTagger(patterns)

# lexicon backed up by the rule-based tagger
lexicon_tagger = nltk.UnigramTagger(train_set, backoff=rule_based_tagger)



In [26]:
# now we will apply rule based modification to viterbi algo

def Viterbi_rule_based(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1] 
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        if pmax!=0.0:
            # getting state for which probability is maximum
            state_max = T[p.index(pmax)]
        else:
            # assigning the unknown tag through rules based 
#             print(lexicon_tagger.tag([word]))
            state_max=lexicon_tagger.tag([word])[0][1]
        state.append(state_max)
    return list(zip(words, state))




In [27]:
# lets check on test sample sentences 
tagged_seq_rule_baes=Viterbi_rule_based(words)
print(tagged_seq_rule_baes)

[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.'), ('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'NOUN'), ('since', 'ADP'), ('2011', 'NUM'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NUM'), ('.', '.'), ('Google', 'NOUN'), ('and', 'CONJ'), ('Twitter', 'NOUN'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'NUM'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'NOUN'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'NOUN'), ("'s", 'PRT'), ('firehose', 'NOUN'), ('.', '.'), ('Twitter', 'NOUN'), ('is', 'VERB'), ('an', 'DET'), ('online', 'NOUN'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DET'), (

So now based on the rules many words are being tagged correctly like
Android , OS , Google , 2013 , personality and many more unknow tags

**Despite of course being a very bad estimate of the true probability, zero-probabilities like this will also result in that
all possible state-sequences for an observation sequence containing an unknown
word will have probability 0 and therefore we cannot choose between them.
So for unknows words we will assign emission probability to 0.001**

In [28]:
# modified viterbi algo with laplace smoothing
# Viterbi Heuristic
def Viterbi_laplace(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]  or 0.001
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
#         print("pmax {0} for word {1}".format(pmax,word))
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))



In [29]:
# lets check on test sample sentences 
tagged_seq_laplace=Viterbi_laplace(words)
print(tagged_seq_laplace)

[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'NOUN'), ('operating', '.'), ('system', 'DET'), ('developed', 'NOUN'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.'), ('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'NOUN'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'NOUN'), ('since', 'ADP'), ('2011', 'NOUN'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'DET'), ('since', 'NOUN'), ('2013', 'NOUN'), ('.', '.'), ('Google', 'NOUN'), ('and', 'CONJ'), ('Twitter', 'NOUN'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'NOUN'), ('that', 'ADP'), ('gave', 'NOUN'), ('Google', 'NOUN'), ('access', '.'), ('to', 'PRT'), ('Twitter', 'VERB'), ("'s", 'PRT'), ('firehose', 'VERB'), ('.', '.'), ('Twitter', 'NOUN'), ('is', 'VERB'), ('an', 'DET'), ('online', 'NOUN'), ('news', '.'), ('and', 'CONJ'), ('social', 'NOUN'), ('networking', '.'), ('service', 'DET'), ('on', 'NOUN'), ('which', 'DET'), ('users'

#### Evaluating tagging accuracy on test data

In [30]:
# lets evaluate tagging accuracy for vanila viterbi algo
tagged_seq_vanila = Viterbi(test_tagged_words)

# accuracy
check = [i for i, j in zip(tagged_seq_vanila, test_run_base) if i == j] 

accuracy_1 = len(check)/len(tagged_seq_vanila)
accuracy_1

0.9214285714285714

In [31]:
# lets evaluate tagging accuracy for modified viterbi with POS tagging
tagged_seq_pos = Viterbi_POS(test_tagged_words)

# accuracy
check = [i for i, j in zip(tagged_seq_pos, test_run_base) if i == j] 

accuracy_2 = len(check)/len(tagged_seq_pos)
accuracy_2

0.95

In [32]:
# lets evaluate tagging accuracy for modified viterbi with rule based tagging
tagged_seq_rule_based = Viterbi_rule_based(test_tagged_words)

# accuracy
check = [i for i, j in zip(tagged_seq_rule_based, test_run_base) if i == j] 

accuracy_3 = len(check)/len(tagged_seq_rule_based)
accuracy_3

0.9857142857142858

In [33]:
# lets evaluate tagging accuracy for modified viterbi algo with laplace smoothing
tagged_seq_laplace = Viterbi_laplace(test_tagged_words)

# accuracy
check = [i for i, j in zip(tagged_seq_laplace, test_run_base) if i == j] 

accuracy_4 = len(check)/len(tagged_seq_laplace)
accuracy_4

0.6785714285714286

### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

vanila viterbi algo tagging accuracy - 0.9214285714285714 <br>
modified viterbi algo with POS tagging - 0.95 <br>
modified vierbi algo with rule based tagging - 0.9857142857142858 <br>
modified tagging with laplace smoothing - 0.6785714285714286 <br>
 It is clear that modified vierbi algo with rule based tagging has best accuracies

### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

### This is pos tagging by vanila viterbi algo

('Android', '.'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', '.'), ('.', '.'), ('Android', '.'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', '.'), ('worldwide', '.'), ('on', 'ADP'), ('smartphones', '.'), ('since', 'ADP'), ('2011', '.'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', '.'), ('.', '.'),

### This is pos tagging by modified viterbi algo

('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.'), ('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'NOUN'), ('since', 'ADP'), ('2011', 'NUM'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NUM'),

We can compare that unknow words like Android , Google ,2013 , smartphones, OS words have been tagged correctly